# Simulating lightcurves for SNe IIP

This notebooks shows an example of a simulating SN IIP lightcurves for 90 days of ZTF observations. (The survey is restricted to 90 days in order to run reasonably fast. A similar simulation for a 3-year survey can be generated using SNIIP_3yr.py.)

In [3]:
import warnings
## No annoying warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import simsurvey.cadence as simul
import sncosmo

import simsurvey_tools as sst

In [4]:
# Set the time frame for the survey (90 days)
mjd_range = (58000, 58090)

The transient generator randomly draws all parameters needed to simulate the lightcurves. For the SNe IIP, one can use one of the built-in sources of `sncosmo` to create a model. In addition, one needs to define a function the randomizes the parameters (peak magnitude and host $E(B-V)$ in this case) and a rate function. 

Further required input is the time frame, which should be set to include a number of days before and after the survey in order to simulate all SNe that may be visible during the observations. Additionally, to reduce the computation time, the  coordinate ranges can be restricted such that only few SNe outside the observed part of the sky are simulated.

In [8]:
dust = sncosmo.CCM89Dust()
model = sncosmo.Model(source='s11-2005lc',
                      effects=[dust],
                      effect_names=['host'],
                      effect_frames=['rest'])

def random_parameters(redshifts, model,
                      mag=(-19.3, 0.1),
                      r_v=2., ebv_rate=0.11,
                      **kwargs):
    # Amplitude
    amp = []
    for z in redshifts:
        model.set(z=z)
        mabs = np.random.normal(mag[0], mag[1])
        model.set_source_peakabsmag(mabs, 'bessellb', 'vega')
        amp.append(model.get('amplitude'))

    return {
        'amplitude': np.array(amp),
        'hostr_v': r_v * np.ones(len(redshifts)),
        'hostebv': np.random.exponential(ebv_rate, len(redshifts))
    }

transientprop = dict(lcmodel=model,
                    lcsimul_func=random_parameters,
                    lcsimul_prop=dict(mag=(-16.75, 0.98)))

tr = simul.get_transient_generator([0.0, 0.1], ratekind='custom',
                                   ratefunc=lambda z: 1.5e-4,
                                   dec_range=[-40,90],
                                   mjd_range=[mjd_range[0] - 90,
                                              mjd_range[1] + 25],
                                   transientprop=transientprop)

The plan object collects all required information of the survey strategy. The function used below generates a simple mock survey to get a basic idea of what we can expect from ZTF, see its docstring in `simsurvey_tools.py` for more information. The output of Eric's `ztf_sim` code can also be loaded into a plan object, see `plan_from_ztf_sim.ipynb`. (Currently the code uses DES $gri$ filters instead of the ZTF ones, which be included later.)

In [9]:
plan = sst.get_survey_plan_simple(mjd_range=mjd_range)

In [10]:
plan.cadence

band,skynoise,field,RA,time,Dec
str4,float64,int64,float64,float64,float64
desr,800.0,661,57.69565,58000.0,31.75385
desr,800.0,666,96.82609,58000.0005208,31.75385
desr,800.0,662,65.52174,58000.0010417,31.75385
desr,800.0,659,42.04348,58000.0015625,31.75385
desr,800.0,658,34.21739,58000.0020833,31.75385
desr,800.0,667,104.65217,58000.0026042,31.75385
desr,800.0,665,89.0,58000.003125,31.75385
desr,800.0,654,2.91304,58000.0036458,31.75385
desr,800.0,656,18.56522,58000.0041667,31.75385


Lastly, some basic properties of the instrument used in the survey must be defined but these are mostly chosen in order to get specific depths of the observations and need not be changed.

In [11]:
instprop = {"desg":{"gain":1.,"zp":30,"zpsys":'ab'},
            "desr":{"gain":1.,"zp":30,"zpsys":'ab'},
            "desi":{"gain":1.,"zp":30,"zpsys":'ab'}}

These three components are collected in a SimulSurvey object, which is then used to generate the lightcurves.

In [12]:
survey = simul.SimulSurvey(generator=tr, 
                           plan=plan, 
                           instprop=instprop)

lcs = survey.get_lightcurves(
    # This cell may give an error because of the progress bar widget.
    # Just delete the following line to fix this.
    progress_bar=True, notebook=True
)

Determining field IDs for all objects

Generating lightcurves



The output is a LightcurveCollection object that organizes the lightcurves and can be accessed as shown below.

In [13]:
lcs[0]

time,band,flux,fluxerr,zp,zpsys
float64,str4,float64,float64,int64,str2
58057.0166667,desr,7788.07527983,804.611662288,30,ab
58057.0302083,desr,7511.91666417,804.611662288,30,ab
58057.1,desr,9018.39522797,804.611662288,30,ab
58057.1135417,desr,6964.17817205,804.611662288,30,ab
58057.1833333,desg,4378.83817365,802.575847293,30,ab
58057.196875,desg,4031.44631435,802.575847293,30,ab
58057.225,desg,3323.4155709,802.575847293,30,ab
58057.2385417,desg,4822.99955661,802.575847293,30,ab
58057.2666667,desi,7091.43102128,1262.95439829,30,ab
